# Import

In [1]:

import json
import os
import re
import time
from typing import List, Optional

import chromadb
import httpx
import pandas as pd
from dotenv import load_dotenv
from pydantic import BaseModel
from rich.console import Console
from rich.table import Table
from tenacity import retry, stop_after_attempt, wait_exponential

## Variables et Configuration

Centralisation de toutes les configurations pour faciliter l'évolution du projet. Modifiez ces variables pour adapter le comportement sans toucher au code.

In [2]:
load_dotenv()

# Clé API
MISTRAL_API_KEY = os.getenv("MISTRAL_API_KEY")

# Configuration générale
CONFIG = {
    "model": "mistral-small-2506",
    "base_url": "https://api.mistral.ai/v1",
    "chat_endpoint": "/chat/completions",
    "embed_model": "mistral-embed",
    "chroma_path": "./chroma_db",
    "rag_n_results": 3,
    "default_temperature": 0.2,
    "default_max_tokens": 512,
    "retry_attempts": 3,
    "retry_wait_min": 10,
    "retry_wait_max": 60,
    "sample_docs": [
        "Paris est la capitale de la France, avec une population d'environ 2,2 millions d'habitants.",
        "Lyon est la troisième plus grande ville de France, connue pour sa gastronomie et son histoire.",
        "Le train TGV relie Paris à Lyon en environ 2 heures.",
        "La Tour Eiffel est un monument emblématique de Paris, construit en 1889.",
        "Mistral AI est une entreprise française spécialisée dans l'IA, fondée en 2023."
    ]
}

### Wrapper API Mistral

In [3]:

class MistralClient:
    def __init__(self):
        self.api_key = MISTRAL_API_KEY
        self.model = CONFIG["model"]
        self.base_url = CONFIG["base_url"]

    @retry(stop=stop_after_attempt(CONFIG["retry_attempts"]),
           wait=wait_exponential(multiplier=1, min=CONFIG["retry_wait_min"], max=CONFIG["retry_wait_max"]))
    def _request(self, endpoint, payload):
        headers = {"Authorization": f"Bearer {self.api_key}", "Content-Type": "application/json"}
        url = f"{self.base_url}{endpoint}"
        response = httpx.post(url, json=payload, headers=headers, timeout=30)
        response.raise_for_status()
        return response.json()

    def chat_completion(self, messages, temperature=None, max_tokens=None, **kwargs):
        if temperature is None:
            temperature = CONFIG["default_temperature"]
        if max_tokens is None:
            max_tokens = CONFIG["default_max_tokens"]
        payload = {
            "model": self.model,
            "messages": messages,
            "temperature": temperature,
            "max_tokens": max_tokens,
            **kwargs
        }
        response = self._request(CONFIG["chat_endpoint"], payload)
        return response["choices"][0]["message"]["content"]

    def completion(self, prompt, temperature=None, max_tokens=None, **kwargs):
        if temperature is None:
            temperature = CONFIG["default_temperature"]
        if max_tokens is None:
            max_tokens = CONFIG["default_max_tokens"]
        payload = {
            "model": self.model,
            "prompt": prompt,
            "temperature": temperature,
            "max_tokens": max_tokens,
            **kwargs
        }
        response = self._request("/completions", payload)
        return response["choices"][0]["text"]

    def embeddings(self, input_texts: List[str]) -> List[List[float]]:
        """
        Génère des embeddings pour une liste de textes en utilisant l'API Mistral.

        Args:
            input_texts (list): Liste de textes à embedder.

        Returns:
            list: Liste des embeddings.
        """
        payload = {
            "model": CONFIG["embed_model"],
            "input": input_texts
        }
        response = self._request("/embeddings", payload)
        return [emb["embedding"] for emb in response["data"]]


In [4]:
SCRATCHPAD_FORMAT = """
Utilisez le scratchpad suivant pour raisonner étape par étape :

Scratchpad :
- Étape 1 : Analyser le problème et identifier les éléments clés.
- Étape 2 : Décomposer le problème en sous-tâches logiques.
- Étape 3 : Résoudre chaque sous-tâche en utilisant des faits ou des raisonnements.
- Étape 4 : Synthétiser les résultats et vérifier la cohérence.

Réponse finale :
"""


def format_prompt_with_scratchpad(user_query):
    """
    Formate un prompt utilisateur avec le scratchpad structuré.
    """
    return f"{SCRATCHPAD_FORMAT}\n\nQuestion : {user_query}"


## Implémentation du Planner

Le planner prend la requête utilisateur et la décompose en une liste d'étapes logiques pour résoudre le problème. Cela suit le pattern décrit dans le guide pour une décomposition automatique.


In [5]:

def planner(query):
    """
    Décompose la tâche en étapes numérotées courtes en utilisant l'API Mistral.

    Args:
        query (str): La requête utilisateur.

    Returns:
        list: Liste des étapes du plan.
    """
    prompt = f"""Tu es un planner. Décompose la tâche en étapes numérotées courtes.

Tâche: {query}

Réponds uniquement avec du JSON valide, sans texte supplémentaire : {{"plan": ["étape 1", "étape 2", ...]}}

Assure-toi que les étapes sont logiques et séquentielles.
"""
    client = MistralClient()
    messages = [{"role": "user", "content": prompt}]
    raw = client.chat_completion(messages, temperature=0.0)
    try:
        # Essayer de parser directement
        return json.loads(raw)["plan"]
    except json.JSONDecodeError:
        # Essayer d'extraire JSON du texte
        json_match = re.search(r'\{.*\}', raw, re.DOTALL)
        if json_match:
            try:
                return json.loads(json_match.group())["plan"]
            except:
                pass
        # Fallback : plan simple
        return ["Analyser la tâche", "Décomposer en étapes", "Résoudre chaque étape", "Synthétiser le résultat"]


## Outils disponibles

Définition des outils que le modèle peut appeler via des actions. Cela inclut un calculateur simple et une recherche mock. Pour un environnement de production, ajoutez du sandboxing pour l'exécution de code.


In [6]:

def execute_tool(action):
    """
    Exécute une action d'outil basée sur la commande reçue.

    Args:
        action (str): La commande d'action, e.g., "CALC: 2+2" ou "SEARCH: query".

    Returns:
        str: L'observation résultante.
    """
    if action.startswith("CALC:"):
        expr = action[5:].strip()
        try:
            # Utilisation d'eval avec un environnement sécurisé (seulement opérations mathématiques de base)
            safe_dict = {
                "__builtins__": {},
                "abs": abs,
                "min": min,
                "max": max,
                "sum": sum,
                "pow": pow,
                "round": round
            }
            result = eval(expr, safe_dict, {})
            return f"OBSERVATION: {result}"
        except (ValueError, SyntaxError, NameError, TypeError) as e:
            return f"OBSERVATION: Erreur dans le calcul - {str(e)}"
    elif action.startswith("SEARCH:"):
        query = action[7:].strip()
        # Implémentation RAG réelle
        results = rag_search(query)
        return f"OBSERVATION: {results}"
    else:
        return "OBSERVATION: Action inconnue"


## Implémentation RAG avec ChromaDB

Configuration de la base de données vectorielle pour la recherche augmentée par récupération (RAG). Utilise ChromaDB avec des embeddings Mistral.


In [7]:

class MistralEmbeddingFunction:
    def __init__(self):
        self.client = MistralClient()

    def __call__(self, input: List[str]) -> List[List[float]]:
        return self.client.embeddings(input)

    def embed_documents(self, texts: List[str]) -> List[List[float]]:
        """Génère des embeddings pour des documents."""
        return self.client.embeddings(texts)

    def embed_query(self, input: List[str]) -> List[List[float]]:
        """Génère un embedding pour une requête unique."""
        return self.client.embeddings(input)

    def name(self):
        return "mistral_embed"


# Initialisation de ChromaDB
chroma_client = chromadb.PersistentClient(path=CONFIG["chroma_path"])
collection = chroma_client.get_or_create_collection(
    name="knowledge_base",
    embedding_function=MistralEmbeddingFunction()
)

# Ajout de documents d'exemple (remplacez par vos propres documents)
sample_docs = CONFIG["sample_docs"]

collection.add(
    documents=sample_docs,
    ids=[f"doc_{i}" for i in range(len(sample_docs))]
)


def rag_search(query, n_results=None):
    """
    Effectue une recherche RAG en utilisant ChromaDB.

    Args:
        query (str): La requête de recherche.
        n_results (int): Nombre de résultats à retourner.

    Returns:
        str: Résultats concaténés.
    """
    if n_results is None:
        n_results = CONFIG["rag_n_results"]
    results = collection.query(query_texts=[query], n_results=n_results)
    docs = results["documents"][0]
    return " ".join(docs)

## Exécution des Étapes

Cette section définit la fonction pour exécuter chaque étape du plan, en utilisant le pattern ReAct pour raisonner, appeler des outils et collecter des observations.

In [8]:
def run_step(step, context):
    """
    Exécute une étape donnée avec le contexte fourni.

    Args:
        step (str): La description de l'étape.
        context (dict): Le contexte incluant le scratchpad précédent.

    Returns:
        dict: Le résultat de l'étape avec pensée, action, observation et résultat.
    """
    prompt = f"""Tu es un assistant de raisonnement utilisant le pattern ReAct.

Pour cette étape: {step}

Contexte précédent: {json.dumps(context, ensure_ascii=False)}

Pense à ce que tu dois faire. Si tu as besoin d'un outil, spécifie l'action à prendre (par exemple: CALC: 2+2 ou SEARCH: population de Paris). Sinon, réponds directement.

Format de réponse attendu (uniquement la pensée et l'action) :
Pensée: [Ta pensée]
Action: [Ton action ou ta réponse finale]
"""
    client = MistralClient()
    messages = [{"role": "user", "content": prompt}]
    resp = client.chat_completion(messages)

    thought_match = re.search(r"Pensée: (.*)", resp)
    action_match = re.search(r"Action: (.*)", resp)

    thought = thought_match.group(1).strip() if thought_match else ""
    action = action_match.group(1).strip() if action_match else resp

    if action.startswith("CALC:") or action.startswith("SEARCH:"):
        observation = execute_tool(action)
        result = observation
    else:
        observation = ""
        result = action

    return {
        "thought": thought,
        "action": action,
        "observation": observation,
        "result": result
    }


## Verifier

Le verifier vérifie la cohérence et la validité du résultat d'une étape. Pour l'instant, une vérification simple : présence d'un résultat.


In [9]:

def verify(step_out):
    """
    Vérifie si le résultat de l'étape est valide.

    Args:
        step_out (dict): Le dictionnaire de sortie de l'étape.

    Returns:
        bool: True si valide, False sinon.
    """
    return "result" in step_out and step_out["result"].strip() != ""


## Modèles Pydantic

Définition des schémas pour valider les sorties des étapes et du raisonnement.


In [10]:

class StepOutput(BaseModel):
    thought: str
    action: str
    observation: Optional[str] = None
    result: Optional[str] = None


class ReasoningOutput(BaseModel):
    plan: List[str]
    scratchpad: List[StepOutput]
    final_answer: str


## Logging et Métriques

Utilisation de pandas pour stocker les logs des raisonnements et rich pour l'affichage.


In [11]:

# DataFrame pour les métriques
metrics_df = pd.DataFrame(columns=["query", "plan_length", "steps_count", "final_answer_length", "time_taken"])

console = Console()


def log_reasoning(query, result, time_taken):
    """
    Log les métriques d'un raisonnement.

    Args:
        query (str): La requête.
        result (dict): Le résultat du raisonnement.
        time_taken (float): Temps pris en secondes.
    """
    global metrics_df
    new_row = {
        "query": query,
        "plan_length": len(result["plan"]),
        "steps_count": len(result["scratchpad"]),
        "final_answer_length": len(result["final_answer"]),
        "time_taken": time_taken
    }
    metrics_df.loc[len(metrics_df)] = new_row


def display_metrics():
    """
    Affiche les métriques avec rich.
    """
    table = Table(title="Métriques de Raisonnement")
    table.add_column("Requête", style="cyan", no_wrap=True)
    table.add_column("Longueur Plan", justify="right")
    table.add_column("Nombre Étapes", justify="right")
    table.add_column("Longueur Réponse", justify="right")
    table.add_column("Temps (s)", justify="right")

    for _, row in metrics_df.iterrows():
        table.add_row(
            row["query"][:50] + "..." if len(row["query"]) > 50 else row["query"],
            str(row["plan_length"]),
            str(row["steps_count"]),
            str(row["final_answer_length"]),
            f"{row['time_taken']:.2f}"
        )

    console.print(table)


## Perform Reasoning

Rassemble toutes les étapes pour effectuer le raisonnement complet sur une requête utilisateur.


In [12]:

def perform_reasoning(query, max_tokens=None):
    """
    Effectue le raisonnement complet sur une requête utilisateur.

    Args:
        query (str): La requête utilisateur.
        max_tokens (int): Nombre maximum de tokens pour les réponses (optionnel).

    Returns:
        dict: Dictionnaire contenant le plan, le scratchpad et la réponse finale.
    """
    if max_tokens is None:
        max_tokens = CONFIG["default_max_tokens"]
    start_time = time.time()

    # Étape 1 : Planning
    plan = planner(query)
    time.sleep(2)  # Pause pour éviter le rate limiting

    # Initialisation du scratchpad
    scratchpad = []

    # Étape 2 : Exécution des étapes
    for step in plan:
        context = {"scratchpad": scratchpad}
        step_out = run_step(step, context)
        scratchpad.append(step_out)

        # Vérification simple
        if not verify(step_out):
            # Réflexion simple : marquer comme erreur
            step_out["result"] = "Erreur détectée, étape à corriger"
        time.sleep(2)  # Pause entre les étapes

    # Étape 3 : Agrégation de la réponse finale
    final_answer = " ".join([s.get("result", "") for s in scratchpad if s.get("result")])

    end_time = time.time()
    time_taken = end_time - start_time

    result = {
        "plan": plan,
        "scratchpad": scratchpad,
        "final_answer": final_answer
    }

    # Log les métriques
    log_reasoning(query, result, time_taken)

    return result


## Exemples d'utilisation

Voici quelques exemples pour tester la couche de raisonnement. Assurez-vous que votre clé API Mistral est configurée dans le fichier .env.


In [13]:

# Exemple 1 : Calcul mathématique simple
print("Exemple 1 : Calcul de 123 * 47")
result1 = perform_reasoning("Calcule 123 * 47 et explique les étapes.")
print("Plan :", result1["plan"])
print("Réponse finale :", result1["final_answer"])
print("\n" + "=" * 50 + "\n")

# Exemple 2 : Recherche avec RAG
print("Exemple 2 : Recherche d'informations sur Paris")
result2 = perform_reasoning("Trouve des informations sur Paris.")
print("Plan :", result2["plan"])
print("Réponse finale :", result2["final_answer"])
print("\n" + "=" * 50 + "\n")

# Exemple 3 : Tâche plus complexe
print("Exemple 3 : Planification d'un voyage")
result3 = perform_reasoning("Planifie un voyage Paris-Lyon en minimisant le temps.")
print("Plan :", result3["plan"])
print("Réponse finale :", result3["final_answer"])
print("\n" + "=" * 50 + "\n")

# Affichage des métriques
display_metrics()


Exemple 1 : Calcul de 123 * 47
Plan : ['Décomposer 47 en 40 + 7', 'Multiplier 123 par 40', 'Multiplier 123 par 7', 'Additionner les résultats des étapes 2 et 3']
Réponse finale : 47 = 40 + 7 OBSERVATION: 492 OBSERVATION: 861 OBSERVATION: 1353


Exemple 2 : Recherche d'informations sur Paris
Plan : ['1. Définir les informations spécifiques à rechercher (géographie, histoire, culture, etc.)', '2. Utiliser des sources fiables (sites officiels, encyclopédies, guides touristiques)', '3. Rechercher des informations générales sur Paris (localisation, population, etc.)', "4. Explorer l'histoire de Paris (fondation, événements marquants)", '5. Découvrir les monuments et lieux emblématiques', '6. Identifier les aspects culturels (musées, festivals, gastronomie)', '7. Vérifier les informations pour leur exactitude', '8. Organiser les données collectées par thème', '9. Compléter avec des détails pratiques (transports, hébergement, etc.)', '10. Synthétiser les informations en un résumé clair']
Répo

                                             Métriques de Raisonnement                                             
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━┓
┃                                                       ┃     Longueur ┃               ┃     Longueur ┃           ┃
┃ Requête                                               ┃         Plan ┃ Nombre Étapes ┃      Réponse ┃ Temps (s) ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━┩
│ Calcule 123 * 47 et explique les étapes.              │            4 │             4 │           63 │     36.41 │
│ Trouve des informations sur Paris.                    │           10 │            10 │         2553 │     86.57 │
│ Planifie un voyage Paris-Lyon en minimisant le tem... │            8 │             8 │         2023 │    124.45 │
└───────────────────────────────────────────────────────┴──────────────┴───────────────┴──────────────┴───────────┘

## Tests avec pytest

Exécution de tests unitaires pour valider les composants du raisonnement.


In [14]:

# Tests simples (exécutez avec pytest dans un terminal séparé)
def test_planner():
    plan = planner("Calcule 2+2")
    assert isinstance(plan, list)
    assert len(plan) > 0


def test_execute_tool_calc():
    obs = execute_tool("CALC: 2+2")
    assert obs == "OBSERVATION: 4"


def test_execute_tool_search():
    obs = execute_tool("SEARCH: Paris")
    assert "OBSERVATION:" in obs


def test_verify():
    step_out = {"result": "some result"}
    assert verify(step_out) == True
    step_out_empty = {"result": ""}
    assert verify(step_out_empty) == False


# Exécuter les tests
if __name__ == "__main__":
    test_planner()
    test_execute_tool_calc()
    test_execute_tool_search()
    test_verify()
    print("Tous les tests passent !")


Tous les tests passent !
